In [1]:
from parsel import Selector
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, InvalidSelectorException
import numpy as np
import pandas as pd
import random
from selenium.webdriver.common.proxy import Proxy, ProxyType


In [2]:
prox = Proxy()
prox.proxy_type = ProxyType.MANUAL
prox.http_proxy = "110.139.112.157:1080"

# prox.socks_proxy = "ip_addr:port"
# prox.ssl_proxy = "ip_addr:port"

capabilities = webdriver.DesiredCapabilities.CHROME
prox.add_to_capabilities(capabilities)

driver = webdriver.Chrome(desired_capabilities=capabilities, service=Service("C:\\chromedriver.exe"))

driver.maximize_window()
driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

driver.find_element(By.ID, "username").send_keys('@')
driver.find_element(By.ID, "password").send_keys('')
driver.find_element(By.XPATH, '//*[@class="btn__primary--large from__button--floating"]').click()

In [104]:
def get_elem(xpath,driver=driver):
    try:
        elem = driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        elem = np.NaN
    return elem

def get_elems(parent_items, xpaths, parent_name):
    items = dict()
    for index,item in enumerate(parent_items):
        elems = dict()
        for elem in xpaths:
            elems.update({elem : get_elem_text(xpaths[elem], driver=item)})
        items.update({f"{parent_name}_{index+1}" : elems})    
    return items

def get_elem_item(parent,item_index):
    try:
        item = parent[item_index].text
    except IndexError:
        item = np.NaN
    return item

def get_elem_text(xpath,driver=driver):
    try:
        elem = driver.find_element(By.XPATH, xpath).text
    except NoSuchElementException:
        elem = np.NaN
    return elem

In [4]:
df_samples = pd.read_excel(r'C:\Users\arman\Downloads\linkedin.xlsx', engine='openpyxl')
fiveLinks=df_samples.iloc[:20, 0]

In [106]:
items = []

for i in fiveLinks:
    driver.get(i)
    
    #clicking buttons
    try:
        get_elem('//*[contains(text(), "more experience")]').click()
        get_elem('//*[@id="ember512"]/div/span/button').click()
    except AttributeError:
            pass
    
    exps = driver.find_elements(By.XPATH, '//*[@class="pv-entity__position-group-pager pv-profile-section__list-item ember-view"]')
    schools = driver.find_elements(By.XPATH, '//*[@class="pv-profile-section__list-item pv-education-entity pv-profile-section__card-item ember-view"]')
    licenses = driver.find_elements(By.XPATH, '//*[@class="pv-profile-section__sortable-item pv-certification-entity ember-view"]')
    volunteer = driver.find_elements(By.XPATH, '//*[@class="pv-profile-section__list-item pv-volunteering-entity pv-profile-section__card-item ember-view"]')
    skills = driver.find_elements(By.XPATH, '//span[@class="pv-skill-category-entity__name-text t-16 t-black t-bold"]')
    
    exps_xpaths = {
        'exp_positions' : './/h3[@class="t-16 t-black t-bold"]',
        'exp_companies' : './/p[@class="pv-entity__secondary-title t-14 t-black t-normal"]',
        'exp_years':  './/span[text()="Dates Employed"]//following::span[1]',
        'exp_durations' : './/span[text()="Employment Duration"]/following::span[1]'
        }
    
    schools_xpaths = {
    
        'school_names' : './/h3[@class="pv-entity__school-name t-16 t-black t-bold"]',
        'degree_name' : './/span[text()="Degree Name"]//following::span[1]',
        'study_field' : './/span[text()="Field Of Study"]//following::span[1]',
        'grade' : './/span[text()="Grade"]//following::span[1]',
        'school_years' : './/span[text()="Dates attended or expected graduation"]//following::span[1]',
        'activities' : './/span[text()="Activities and Societies:"]//following::span[1]'
        }
    
    licenses_xpaths = {
        'licenses' : './/h3[@class="t-16 t-bold"]',
        'issue_date' : './/span[text()="Issued date and, if applicable, expiration date of the certification or license"]//following::span[1]',
        'issue_authority' :'.//span[text()="Issuing authority"]//following::span[1]'
        }

    item = {
        
        #left header
        'name' : get_elem_text('//*[@class="text-heading-xlarge inline t-24 v-align-middle break-words"]'), 
        'job_title' : get_elem_text('//*[@class="text-body-medium break-words"]'), 
        'small_bio' : get_elem_text('//*[@class="text-body-small t-black--light break-words pt1"]'), 
        'loc' : get_elem_text('//*[@class="text-body-small inline t-black--light break-words"]'),  
    
        #right header
        'cur_comp' : get_elem_text('//*[@aria-label="Current company"]'), 
        'educ' : get_elem_text('//*[@aria-label="Education"]'), 
        
        #experiences 
        'experiences' : get_elems(exps, exps_xpaths,'company'),
        
        #schools
        'schools' : get_elems(schools,schools_xpaths, 'university'),        
        
        #licenses
        'licenses' : get_elems(licenses, licenses_xpaths, "license"),

        #about
        'about' : get_elem_text('//*[@class="pv-profile-section pv-about-section artdeco-card p5 mt4 ember-view"]'),
        'skills' : get_elem_item(skills,0)
    }
        
    items.append(item)
    
    sleep(abs(np.random.normal(7, 2, size=(1, )).item()))

driver.quit()

In [107]:
data = pd.json_normalize(items,max_level=2)

In [108]:
data.to_csv("items.csv")